# Práctica de Bioinformática - Sesión 04
## Uso de BLAST con Biopython

Autores: **Raúl Mendoza**, **Adrián Ojeda**  
Asignatura: **Bioinformática (ULPGC)**

En esta práctica trabajamos con BLAST (Basic Local Alignment Search Tool) para buscar secuencias similares en bases de datos biológicas, usando **Biopython** tanto en modo online (servidores de NCBI) como en modo local (BLAST+ instalado en nuestra máquina).

> Nota importante
>
> En este cuaderno programamos todas las soluciones, pero muchas celdas que hacen consultas reales a BLAST NO se ejecutan aquí porque necesitan conexión a internet y/o > tener BLAST+ instalado con sus bases de datos.

## Requisitos previos

- Python 3.x
- Paquete **Biopython** instalado:
  ```bash
  pip install biopython
  ```
- Para la parte **online**: conexión a internet y una dirección de correo (NCBI la pide por cortesía para identificar al usuario).
- Para la parte **local**:
  - Tener **BLAST+** instalado (ejecutables como `blastn`, `blastp`, `blastx`, `tblastn`,     `tblastx`).
  - Tener al menos una base de datos BLAST ya creada (por ejemplo con `makeblastdb`).
  - Tener el directorio de los ejecutables en el `PATH` o indicar la ruta completa.

In [4]:
from Bio.Blast import NCBIWWW, NCBIXML
import subprocess

from Bio import SeqIO
import subprocess
import os
import tempfile
import statistics


### Funciones auxiliares comunes

Antes de meternos con los ejercicios, defino algunas funciones que reutilizo varias veces: escribir una secuencia a FASTA temporal, calcular porcentaje de identidad y cobertura, etc.

In [5]:
def escribir_fasta_temporal(seq, descripcion="query"):
    """Guarda la secuencia en un fichero FASTA temporal y devuelve la ruta.

    Esto es útil para llamar a BLAST local, que normalmente espera un archivo.
    """
    tmp = tempfile.NamedTemporaryFile(delete=False, suffix=".fasta", mode="w")
    tmp.write(f">{descripcion}\n")
    seq_limpia = ''.join(seq.split()).upper()
    tmp.write(seq_limpia + "\n")
    tmp.close()
    return tmp.name


def porcentaje_identidad(hsp):
    """Devuelve el porcentaje de identidad de un HSP."""
    if hsp.align_length == 0:
        return 0.0
    return 100.0 * hsp.identities / hsp.align_length


def cobertura_query(hsp, query_length):
    """Porcentaje de la query cubierta por este HSP."""
    if query_length == 0:
        return 0.0
    return 100.0 * hsp.align_length / query_length


## Ejercicio 1

Escribe un programa en Python que use Biopython para hacer una búsqueda BLAST de una secuencia de ADN que introduzcas por teclado. El programa debe mostrar por pantalla:
- el número de resultados obtenidos,
- el E-value del mejor resultado,
- y la descripción de la secuencia más similar.
Hágalo de forma online y local.


### 1.a) Búsqueda BLASTN online

Usamos `NCBIWWW.qblast` para enviar la secuencia al servidor BLAST de NCBI y parseamos el XML devuelto con `NCBIXML`.

In [ ]:
from Bio.Blast import NCBIWWW, NCBIXML

def blastn_online_desde_teclado_mejorado():
    secuencia = input("Introduce la secuencia de ADN (sin espacios): ")
    secuencia = ''.join(secuencia.split()).upper()

    if not secuencia:
        print("Error: No has introducido ninguna secuencia.")
        return

    print(f"\n--- Conectando con NCBI (Base de datos: nt) ---")
    print(f"Longitud de tu consulta: {len(secuencia)} nucleótidos")
    print("Espere, esto puede tardar unos segundos o minutos...")

    try:
        result_handle = NCBIWWW.qblast(
            program="blastn",
            database="nt",
            sequence=secuencia,
            expect=0.05,       
            hitlist_size=50,   
            megablast=True
        )

        record = NCBIXML.read(result_handle)
        result_handle.close()

        num_hits = len(record.alignments)
        print(f"\nResultados obtenidos: {num_hits} hits")

        if num_hits == 0:
            print("No se encontraron coincidencias significativas.")
            return

        best_alignment = record.alignments[0]
        best_hsp = best_alignment.hsps[0] 

        identidad = (best_hsp.identities / best_hsp.align_length) * 100
        cobertura = (best_hsp.align_length / record.query_length) * 100

        print("-" * 40)
        print("MEJOR ALINEAMIENTO ENCONTRADO:")
        print(f"Descripción: {best_alignment.hit_def}")
        print(f"ID Acceso:   {best_alignment.hit_id}")
        print(f"E-value:     {best_hsp.expect}")
        print(f"Bit Score:   {best_hsp.bits}")
        print(f"Identidad:   {identidad:.2f}%")
        print(f"Cobertura:   {cobertura:.2f}%")
        print("-" * 40)

    except Exception as e:
        print(f"\nOcurrió un error de conexión o procesamiento: {e}")

blastn_online_desde_teclado_mejorado()

#Secuencia de prueba para BLASTN. AGCCCTCCAGGACAGGCTGCATCAGAAGAGGCCATCAAGCAGGTCTGTTCCAAGGGCCTTTGCGTCAGGTGGGCTCAGGATTCCAGGGTGGCTGGACCCCAGGCCCCAGCTCTGCAGCAGGGAGGACGTGGCTGGGCTCGTGAAGCATGTGGGGGTGAGCCCAGGGGCCCCAAGGCAGGGCACCTGGCCTTCAGCCTGCCTCAGCCCTGC


--- Conectando con NCBI (Base de datos: nt) ---
Longitud de tu consulta: 210 nucleótidos
Espere, esto puede tardar unos segundos o minutos...

Resultados obtenidos: 50 hits
----------------------------------------
MEJOR ALINEAMIENTO ENCONTRADO:
Descripción: Homo sapiens insulin isoform U2 (INS) mRNA, complete cds, alternatively spliced
ID Acceso:   gi|1883396563|gb|MT335688.1|
E-value:     1.3994e-103
Bit Score:   388.917
Identidad:   100.00%
Cobertura:   100.00%
----------------------------------------


### 1.b) Búsqueda BLASTN local

Ahora hacemos lo mismo, pero usando BLAST+ instalado en local. Asumimos que:
- `blastn` está en el PATH,
- existe una BD local de nucleótidos, por ejemplo `genomasbase`.

In [ ]:
def blastn_local_desde_teclado(
    db="genomasbase",
    blastn_path="blastn", 
    out_xml="blastn_local_result.xml",
):
    secuencia = input("Introduce la secuencia de ADN (sin espacios): ")
    secuencia = ''.join(secuencia.split()).upper()

    with tempfile.NamedTemporaryFile(delete=False, suffix=".fasta", mode="w") as tmp:
        tmp.write(f">query_local\n{secuencia}\n")
        query_fasta = tmp.name

    comando = f'{blastn_path} -task blastn-short -dust no -query "{query_fasta}" -db "{db}" -out "{out_xml}" -evalue 0.05 -outfmt 5'
    
    print(f"Ejecutando: {comando}")

    try:
        subprocess.run(comando, shell=True, check=True)
        
        with open(out_xml) as handle:
            record = NCBIXML.read(handle)
        
        num_hits = len(record.alignments)
        print(f"Número de hits: {num_hits}")

        if num_hits == 0:
            print("No se encontraron resultados.")
        else:
            best_alignment = record.alignments[0]
            best_hsp = best_alignment.hsps[0]
            print(f"Mejor e-value: {best_hsp.expect}")
            print(f"Descripción: {best_alignment.hit_def}")
            # Agregamos esto para ver qué secuencia encontró
            print(f"Identidad: {best_hsp.identities}/{best_hsp.align_length}")

    except subprocess.CalledProcessError as e:
        print(f"Error al ejecutar BLAST: {e}")
    except Exception as e:
        print(f"Ocurrió un error: {e}")
    finally:
        if os.path.exists(query_fasta):
            os.remove(query_fasta)

blastn_local_desde_teclado()

Ejecutando: blastn -task blastn-short -dust no -query "C:\Users\adrio\AppData\Local\Temp\tmpm93pd_k4.fasta" -db "genomasbase" -out "blastn_local_result.xml" -evalue 0.05 -outfmt 5
Número de hits: 1
Mejor e-value: 9.74213e-33
Descripción: seq3_ejemplo
Identidad: 60/60


## Ejercicio 2

Escribe un programa en Python que use Biopython para hacer una búsqueda BLAST de una secuencia de proteína que introduzcas por teclado. El programa debe guardar en un fichero los resultados que tengan un E-value menor que 0.001, incluyendo: id, longitud, E-value y porcentaje de identidad. Hágalo de forma online y local.


### 2.a) BLASTP online con filtrado por E-value


In [ ]:
from Bio.Blast import NCBIWWW, NCBIXML

def blastp_online_filtrado_mejorado(
    salida="blastp_online_filtrado.txt",
    evalue_umbral=0.001,
):
    secuencia = input("Introduce la secuencia de proteína: ").strip().upper()
    
    if not secuencia:
        print("Error: La secuencia no puede estar vacía.")
        return

    print(f"\n--- Conectando con NCBI (BLASTP - Base de datos: nr) ---")
    print("Espere mientras buscamos proteínas similares...")

    try:
        result_handle = NCBIWWW.qblast(
            program="blastp",
            database="nr",
            sequence=secuencia,
            expect=10.0,      
            hitlist_size=50   
        )

        record = NCBIXML.read(result_handle)
        result_handle.close()

        hits_guardados = 0

        with open(salida, "w") as f:
            f.write("Identificador\tLongitud\tE-value\t%_Identidad\n")

            for alignment in record.alignments:
                for hsp in alignment.hsps:
                    if hsp.expect < evalue_umbral:
                        
                        identidad = (hsp.identities / hsp.align_length) * 100
                        
                        f.write(
                            f"{alignment.hit_id}\t"
                            f"{alignment.length}\t"
                            f"{hsp.expect}\t"
                            f"{identidad:.2f}\n"
                        )
                        hits_guardados += 1

        print("-" * 40)
        if hits_guardados > 0:
            print(f"Se han guardado {hits_guardados} resultados en '{salida}'.")
            print("Verifica la carpeta de tu proyecto para ver el archivo.")
        else:
            print(f"Se encontraron alineamientos, pero ninguno cumplió el requisito de E-value < {evalue_umbral}.")

    except Exception as e:
        print(f"\nError al conectar o procesar: {e}")

blastp_online_filtrado_mejorado()

## Secuencia de prueba para BLASTP: MVHLTPEEKSAVTALWGKVNVDEVGGEALGRLLVVYPWTQRFFESFGDLSTPDAVMGNPKVKAHGKKVLGAFSDGLAHLDNLKGTFATLSELHCDKLHVDPENFRLLGNVLVCVLAHHFGKEFTPPVQAAYQKVVAGVANALAHKYH


--- Conectando con NCBI (BLASTP - Base de datos: nr) ---
Espere mientras buscamos proteínas similares...
----------------------------------------
¡Éxito! Se han guardado 99 resultados en 'blastp_online_filtrado.txt'.
Verifica la carpeta de tu proyecto para ver el archivo.


### 2.b) BLASTP local con filtrado por E-value


In [27]:
import subprocess
import os
import tempfile
from Bio.Blast import NCBIXML

def blastp_local_filtrado(
    db="proteinasbase",
    blastp_path="blastp", 
    out_xml="blastp_local.xml",
    salida="blastp_local_filtrado.txt",
    evalue_umbral=0.001,
):
    secuencia = input("Introduce la secuencia de proteína: ")
    secuencia = ''.join(secuencia.split()).upper()

    with tempfile.NamedTemporaryFile(delete=False, suffix=".fasta", mode="w") as tmp:
        tmp.write(f">query_prot_local\n{secuencia}\n")
        query_fasta = tmp.name

    comando = f'{blastp_path} -query "{query_fasta}" -db "{db}" -out "{out_xml}" -evalue 1.0 -outfmt 5'
    
    print(f"Ejecutando: {comando}")

    try:
        subprocess.run(comando, shell=True, check=True)

        with open(out_xml) as handle:
            record = NCBIXML.read(handle)

        hits_guardados = 0
        with open(salida, "w") as f:
            f.write("id\tlongitud\tevalue\tporc_identidad\n")
            
            for alignment in record.alignments:
                for hsp in alignment.hsps:
                    if hsp.expect < evalue_umbral:
                        try:
                            pid = porcentaje_identidad(hsp)
                        except NameError:
                            pid = (hsp.identities / hsp.align_length) * 100
                            
                        f.write(
                            f"{alignment.hit_id}\t{alignment.length}\t"
                            f"{hsp.expect}\t{pid:.2f}\n"
                        )
                        hits_guardados += 1
        
        print(f"Proceso terminado. Se guardaron {hits_guardados} resultados en '{salida}'.")

    except subprocess.CalledProcessError as e:
        print(f"Error crítico al ejecutar BLASTP: {e}")
        print("Verifica que la base de datos 'proteinasbase' existe y fue creada con '-dbtype prot'")
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")
    finally:
        if os.path.exists(query_fasta):
            os.remove(query_fasta)

blastp_local_filtrado()

Ejecutando: blastp -query "C:\Users\adrio\AppData\Local\Temp\tmp25shshpi.fasta" -db "proteinasbase" -out "blastp_local.xml" -evalue 1.0 -outfmt 5
Proceso terminado. Se guardaron 1 resultados en 'blastp_local_filtrado.txt'.


## Ejercicio 3

Escribe un programa que lea una secuencia de ADN desde un fichero FASTA, haga una búsqueda BLAST y filtre los resultados por organismo. Debe mostrar el número de resultados filtrados y el E-value medio. Hágalo online y local.


### 3.a) BLASTN online leyendo la query de un FASTA


In [16]:
def blastn_online_desde_fasta_filtrado_organismo(fasta_path, organismo, evalue_max=10.0):
    from Bio.Blast import NCBIWWW, NCBIXML
    from Bio import SeqIO
    import statistics

    record_fasta = SeqIO.read(fasta_path, "fasta")
    print(f"Buscando para: {organismo}...")

    # MEJORA: Filtramos directamente en el servidor NCBI
    # Esto asegura que los resultados devueltos sean SOLO de ese organismo
    entrez_filter = f"{organismo}[ORGN]" 

    result_handle = NCBIWWW.qblast(
        program="blastn",
        database="nt",
        sequence=record_fasta.seq,
        expect=evalue_max,
        hitlist_size=100,
        entrez_query=entrez_filter  # <--- Agregado aquí
    )

    blast_record = NCBIXML.read(result_handle)
    result_handle.close()

    evalues = []
    for alignment in blast_record.alignments:
        for hsp in alignment.hsps:
            evalues.append(hsp.expect)

    if evalues:
        print(f"Hits encontrados: {len(evalues)}")
        print(f"E-value medio: {statistics.mean(evalues)}")
    else:
        print("No se encontraron resultados para ese organismo.")

### 3.b) BLASTN local leyendo la query de un FASTA


In [21]:
def blastn_local_desde_fasta_filtrado_organismo(
    fasta_path,
    organismo,
    db="genomasbase",
    blastn_path="blastn", 
    out_xml="blastn_local_fasta.xml",
    evalue_max=10.0,
):
    if not os.path.exists(fasta_path):
        print(f"Error: No existe el archivo {fasta_path}")
        return

    comando = f'{blastn_path} -task blastn-short -dust no -query "{fasta_path}" -db "{db}" -out "{out_xml}" -evalue {evalue_max} -outfmt 5'
    
    print(f"Ejecutando: {comando}")

    try:
        subprocess.run(comando, shell=True, check=True)

        with open(out_xml) as handle:
            blast_record = NCBIXML.read(handle)

        organismo_lower = organismo.lower()
        evalues_filtrados = []

        for alignment in blast_record.alignments:
            descripcion = alignment.hit_def.lower()
            

            if organismo_lower in descripcion:
                for hsp in alignment.hsps:
                    evalues_filtrados.append(hsp.expect)

        num_resultados = len(evalues_filtrados)
        print(f"Número de resultados para '{organismo}': {num_resultados}")

        if num_resultados > 0:
            evalue_medio = statistics.mean(evalues_filtrados)
            print(f"E-value medio: {evalue_medio}")
        else:
            print("Se ejecutó BLAST correctamente y encontró hits, pero ninguno coincidía con tu palabra clave.")
            if len(blast_record.alignments) > 0:
                print(f"(Nota: BLAST encontró {len(blast_record.alignments)} hits, pero sus descripciones no contenían '{organismo}')")

    except subprocess.CalledProcessError as e:
        print(f"Error de BLAST: {e}")
    except Exception as e:
        print(f"Error: {e}")

print("\n--- INTENTO CORREGIDO: Buscando 'ejemplo' ---")
blastn_local_desde_fasta_filtrado_organismo("query_ejemplo.fasta", "ejemplo")


--- INTENTO CORREGIDO: Buscando 'ejemplo' ---
Ejecutando: blastn -task blastn-short -dust no -query "query_ejemplo.fasta" -db "genomasbase" -out "blastn_local_fasta.xml" -evalue 10.0 -outfmt 5
Número de resultados para 'ejemplo': 4
E-value medio: 7.310780000000033e-06


## Ejercicio 4

Lleve a cabo una búsqueda de ejemplo con cada una de las cinco utilidades de la suite descritas (`blastn`, `blastp`, `blastx`, `tblastn`, `tblastx`), tanto online como local. Comente qué especies tienen homólogos más cercanos y qué porcentaje de identidad y cobertura se observa.


Primero definimos una función para resumir el mejor hit de un BLAST (o los tres primeros).

In [ ]:
def resumen_mejor_hit(blast_record, max_hits=3):
    if len(blast_record.alignments) == 0:
        print("No se obtuvieron hits.")
        return

    for i, alignment in enumerate(blast_record.alignments[:max_hits], start=1):
        best_hsp = alignment.hsps[0]
        pid = porcentaje_identidad(best_hsp)
        cov = cobertura_query(best_hsp, blast_record.query_length)
        print(f"Hit {i}:")
        print(f"  ID: {alignment.hit_id}")
        print(f"  Descripción: {alignment.hit_def}")
        print(f"  E-value: {best_hsp.expect}")
        print(f"  % identidad: {pid:.2f}")
        print(f"  Cobertura query: {cov:.2f}%")
        print()

### 4.a) BLAST online: blastn, blastp, blastx, tblastn, tblastx


In [ ]:
from Bio.Blast import NCBIWWW, NCBIXML

def lanzar_blastn_online(secuencia_dna):
    result_handle = NCBIWWW.qblast("blastn", "nt", secuencia_dna)
    record = NCBIXML.read(result_handle)
    result_handle.close()
    return record

def lanzar_blastp_online(secuencia_prot):
    result_handle = NCBIWWW.qblast("blastp", "nr", secuencia_prot)
    record = NCBIXML.read(result_handle)
    result_handle.close()
    return record

def lanzar_blastx_online(secuencia_dna):
    result_handle = NCBIWWW.qblast("blastx", "nr", secuencia_dna)
    record = NCBIXML.read(result_handle)
    result_handle.close()
    return record

def lanzar_tblastn_online(secuencia_prot):
    result_handle = NCBIWWW.qblast("tblastn", "nt", secuencia_prot)
    record = NCBIXML.read(result_handle)
    result_handle.close()
    return record

def lanzar_tblastx_online(secuencia_dna):
    result_handle = NCBIWWW.qblast("tblastx", "nt", secuencia_dna)
    record = NCBIXML.read(result_handle)
    result_handle.close()
    return record

# Ejemplo de cómo los usaríamos en nuestra máquina:
# dna_ejemplo = "ATGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG"
# prot_ejemplo = "MKTIIALSYIFCLVFAD"
# rec_n = lanzar_blastn_online(dna_ejemplo); resumen_mejor_hit(rec_n)
# rec_p = lanzar_blastp_online(prot_ejemplo); resumen_mejor_hit(rec_p)
# rec_x = lanzar_blastx_online(dna_ejemplo); resumen_mejor_hit(rec_x)
# rec_tn = lanzar_tblastn_online(prot_ejemplo); resumen_mejor_hit(rec_tn)
# rec_tx = lanzar_tblastx_online(dna_ejemplo); resumen_mejor_hit(rec_tx)


### 4.b) BLAST local para las cinco utilidades


In [ ]:
def porcentaje_identidad(hsp):
    if hsp.align_length == 0: return 0.0
    return 100.0 * hsp.identities / hsp.align_length

def cobertura_query(hsp, query_length):
    if query_length == 0: return 0.0
    return 100.0 * hsp.align_length / query_length

def resumen_mejor_hit(blast_record, programa):
    print(f"\nResultados para: {programa.upper()}")
    num_hits = len(blast_record.alignments)
    print(f"  > Hits encontrados: {num_hits}")

    if num_hits > 0:
        best_aln = blast_record.alignments[0]
        best_hsp = best_aln.hsps[0]
        
        pid = porcentaje_identidad(best_hsp)
        cov = cobertura_query(best_hsp, blast_record.query_length)
        
        print(f"  > Mejor Hit ID: {best_aln.hit_id}")
        print(f"  > Descripción: {best_aln.hit_def[:60]}...") 
        print(f"  > E-value: {best_hsp.expect}")
        print(f"  > % Identidad: {pid:.2f}%")
        print(f"  > Cobertura: {cov:.2f}%")
    else:
        print("  > No se encontraron coincidencias significativas.")
    print("-" * 50)

def ejercicio_4_blast_local_completo(
    dna_seq, 
    prot_seq, 
    db_nucl="genomasbase", 
    db_prot="proteinasbase"
):
    with tempfile.NamedTemporaryFile(delete=False, suffix=".fasta", mode="w") as t_dna:
        t_dna.write(f">query_dna\n{dna_seq}\n")
        dna_fasta = t_dna.name
        
    with tempfile.NamedTemporaryFile(delete=False, suffix=".fasta", mode="w") as t_prot:
        t_prot.write(f">query_prot\n{prot_seq}\n")
        prot_fasta = t_prot.name

    variantes = [
        ("blastn",  dna_fasta,  db_nucl, "-task blastn-short -dust no"), 
        
        ("blastp",  prot_fasta, db_prot, "-task blastp-short"),
        
        ("blastx",  dna_fasta,  db_prot, ""), 
        
        ("tblastn", prot_fasta, db_nucl, ""),
        
        ("tblastx", dna_fasta,  db_nucl, "") 
    ]

    print("=== INICIANDO EJERCICIO 4: 5 VARIANTES DE BLAST LOCAL ===\n")

    for prog, query, db, flags in variantes:
        out_xml = f"{prog}_temp.xml"
        
        comando = f'{prog} -query "{query}" -db "{db}" -out "{out_xml}" -outfmt 5 -evalue 20.0 {flags}'
        
        
        try:
            subprocess.run(comando, shell=True, check=True)
            
            with open(out_xml) as handle:
                try:
                    record = NCBIXML.read(handle)
                    resumen_mejor_hit(record, prog)
                except ValueError:
                    print(f"Resultados para {prog.upper()}: Error leyendo XML (quizás vacío).\n")
                    
        except subprocess.CalledProcessError as e:
            print(f"Error ejecutando {prog}: {e}")
            print(f"Verifica que tienes la base de datos correcta ({db}) creada.\n")
        finally:
            if os.path.exists(out_xml):
                os.remove(out_xml)

    if os.path.exists(dna_fasta): os.remove(dna_fasta)
    if os.path.exists(prot_fasta): os.remove(prot_fasta)
    print("=== FIN DEL EJERCICIO ===")


mi_adn = "ATGCGTACGTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTACGATCGATCGATCGATC"

mi_prot = "MVLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHFDLSHGSAQVKGHG"

ejercicio_4_blast_local_completo(mi_adn, mi_prot)

=== INICIANDO EJERCICIO 4: 5 VARIANTES DE BLAST LOCAL ===


Resultados para: BLASTN
  > Hits encontrados: 1
  > Mejor Hit ID: gnl|BL_ORD_ID|0
  > Descripción: seq1_ejemplo...
  > E-value: 9.74213e-33
  > % Identidad: 100.00%
  > Cobertura: 100.00%
--------------------------------------------------

Resultados para: BLASTP
  > Hits encontrados: 2
  > Mejor Hit ID: gnl|BL_ORD_ID|0
  > Descripción: prot1_ejemplo...
  > E-value: 2.32044e-41
  > % Identidad: 100.00%
  > Cobertura: 100.00%
--------------------------------------------------

Resultados para: BLASTX
  > Hits encontrados: 0
  > No se encontraron coincidencias significativas.
--------------------------------------------------

Resultados para: TBLASTN
  > Hits encontrados: 0
  > No se encontraron coincidencias significativas.
--------------------------------------------------

Resultados para: TBLASTX
  > Hits encontrados: 1
  > Mejor Hit ID: gnl|BL_ORD_ID|0
  > Descripción: seq1_ejemplo...
  > E-value: 4.83544e-10
  > % Identid

---

Con este cuaderno damos por resuelta la práctica de la Sesión 04 de laboratorio.
Hemos programado soluciones para las cuatro tareas: búsquedas BLAST online y locales, filtrado por E-value, filtrado por organismo y ejemplos con las cinco variantes de la suite BLAST.

Autores: Raúl Mendoza, Adrián Ojeda.